In [26]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: ml_playground
Azure region: westeurope
Subscription id: eb2fe91a-2797-4c53-bd12-b644f041e7ef
Resource group: rg-art-ta-cfp-docker


In [27]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "compute-cluster-D2V2"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [28]:
from azureml.core import ScriptRunConfig, Environment
env = Environment.get(workspace=ws, name="AzureML-Tutorial")

In [29]:
from azureml.widgets import RunDetails
#from azureml.train import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C': choice(0.001, 100), '--max_iter': choice(100,800)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
#sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', script='train.py', compute_target=aml_compute, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=20, 
                                     max_concurrent_runs=4, 
                                     max_duration_minutes=10080, 
                                     policy=policy, 
                                     run_config=src)

In [30]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [32]:
import joblib
# Get your best run and save the model from that run.

best = hdr.get_best_run_by_primary_metric()

print("Best run metrics :", best.get_metrics())
print("Best run file names :", best.get_file_names())

model = best.register_model(model_name = 'best_model', model_path='outputs/model.joblib')

Best run metrics : {'Regularization Strength:': 0.001, 'Max iterations:': 100, 'Accuracy': 0.9154779969650987}
Best run file names : ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dataset = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

In [35]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    compute_target=aml_compute,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric= 'accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5)


In [36]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output = True)
automl_run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on compute-cluster-D2V2 with default configuration
Running on remote compute: compute-cluster-D2V2


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_10aaa21b-8fb5-44ae-b4f2-d7ee867c419d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

{'runId': 'AutoML_10aaa21b-8fb5-44ae-b4f2-d7ee867c419d',
 'target': 'compute-cluster-D2V2',
 'status': 'Completed',
 'startTimeUtc': '2023-04-24T14:31:30.936034Z',
 'endTimeUtc': '2023-04-24T15:20:34.122261Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-cluster-D2V2',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"eb2fe91a-2797-4c53-bd12-b644f041e7ef","resource_group":"rg-art-ta-cfp-docker","workspace_name":"ml_playground","region":"westeurope","compute_target":"compute-cluster-D2V2","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_bat

In [17]:
# Retrieve and save your best automl model.

automl_best_run, automl_best_model = automl_run.get_output()

print("Best run metrics :", automl_best_run)

best_automl_model = automl_run.register_model(model_name='best_automl_model_01')

Best run metrics : Run(Experiment: udacity-project,
Id: AutoML_5d3ee554-ba85-496e-8e48-a5811376a638_29,
Type: azureml.scriptrun,
Status: Completed)


In [18]:
# Delete compute cluster

aml_compute.delete()

Current provisioning state of AmlCompute is "Deleting"

